# 📈 Public Time Series Forecasting Tool
Upload your own time series data (CSV with two columns: `ds` and `y`) or use the default sample.
This notebook compares **Prophet** and **LSTM** models for forecasting.

In [ ]:
# Install required libraries
!pip install prophet tensorflow scikit-learn pandas matplotlib --quiet

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
# Upload CSV or use default sample
from google.colab import files
import io
try:
    uploaded = files.upload()
    df = pd.read_csv(io.BytesIO(list(uploaded.values())[0]))
except:
    print('No file uploaded. Using default sample data.')
    months = pd.date_range(start='2022-01-01', periods=24, freq='M')
    sales = [120, 130, 150, 140, 135, 145, 170, 165, 180, 250, 300, 320,
             125, 135, 155, 145, 140, 150, 175, 170, 185, 260, 310, 330]
    df = pd.DataFrame({'ds': months, 'y': sales})

In [ ]:
# Prophet forecasting
prophet = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
prophet.add_seasonality(name='flu_season', period=12, fourier_order=4)
prophet.fit(df)
future = prophet.make_future_dataframe(periods=3, freq='M')
forecast_prophet = prophet.predict(future)

In [ ]:
# LSTM forecasting with 6-month window
data = df['y'].values.reshape(-1, 1)
scaler = MinMaxScaler()
scaled = scaler.fit_transform(data)
X, y = [], []
window = 6
for i in range(window, len(scaled)):
    X.append(scaled[i-window:i])
    y.append(scaled[i])
X, y = np.array(X), np.array(y)
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(window, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=200, verbose=0)

In [ ]:
# Forecast next 3 months using LSTM
input_seq = scaled[-window:].reshape(1, window, 1)
preds = []
for _ in range(3):
    pred = model.predict(input_seq, verbose=0)[0]
    preds.append(pred)
    input_seq = np.append(input_seq[:, 1:, :], [[pred]], axis=1)
lstm_forecast = scaler.inverse_transform(np.array(preds).reshape(-1, 1)).flatten()

In [ ]:
# Plot forecasts
plt.figure(figsize=(12, 6))
plt.plot(df['ds'], df['y'], label='Actual Sales')
plt.plot(forecast_prophet['ds'], forecast_prophet['yhat'], label='Prophet Forecast', linestyle='--')
future_dates = pd.date_range(start=df['ds'].max() + pd.DateOffset(months=1), periods=3, freq='M')
plt.plot(future_dates, lstm_forecast, label='LSTM Forecast', linestyle=':', marker='o')
plt.title('Prophet vs LSTM Forecast')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()